In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [ ]:
documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
response = query_engine.query("what are in the introduction part?")
print(response)

In [6]:
from llama_index.readers.google import GoogleDriveReader

# Initialize the reader
reader = GoogleDriveReader(
    folder_id="1qq125mDOBJ0sNJLh7znrFfFzaQOm7L-m",
    # service_account_key="[SERVICE_ACCOUNT_KEY_JSON]",
    credentials_path="credentials.json",
    cache_path='./my_cache/'
)
# Load data
documents = reader.load_data()

# List resources in the drive
resources = reader.list_resources()

# Get information about a specific resource
# resource_info = reader.get_resource_info("file.txt")

# # Load a specific resource
# specific_doc = reader.load_resource("file.txt")

# # Read file content directly
# file_content = reader.read_file_content("path/to/file.txt")

print(f"Loaded {len(documents)} documents")
print(f"Found {len(resources)} resources")
print(f"Found {documents} resources")
# print(f"Resource info: {resource_info}")
# print(f"Specific document: {specific_doc}")
# print(f"File content length: {len(file_content)} bytes")

2025-09-10 12:12:55,829 - INFO - file_cache is only supported with oauth2client<4.0.0


2025-09-10 12:12:56,848 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-09-10 12:12:58,922 - INFO - file_cache is only supported with oauth2client<4.0.0


Loaded 1 documents
Found 1 resources
Found [Document(id_='1HsRhY1HA8tOd_-eROAVrwiL4NER65KHf', embedding=None, metadata={'page_label': '1', 'file_name': '1HsRhY1HA8tOd_-eROAVrwiL4NER65KHf.pdf', 'file id': '1HsRhY1HA8tOd_-eROAVrwiL4NER65KHf', 'author': 'kenji sugino', 'file path': 'japan visa documents/VISITING US MILITARY PERSONNEL.pdf', 'mime type': 'application/pdf', 'created at': '2025-09-10T01:43:11.251Z', 'modified at': '2025-08-08T17:10:16.000Z'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='VISITING US MIILITALY PERSONNEL \nApril 2025 \nA. PURPOSE \nVisit a US military personnel stationed in Japan or his/her dependent . \n \nB. Re

In [7]:
documents

[Document(id_='1HsRhY1HA8tOd_-eROAVrwiL4NER65KHf', embedding=None, metadata={'page_label': '1', 'file_name': '1HsRhY1HA8tOd_-eROAVrwiL4NER65KHf.pdf', 'file id': '1HsRhY1HA8tOd_-eROAVrwiL4NER65KHf', 'author': 'kenji sugino', 'file path': 'japan visa documents/VISITING US MILITARY PERSONNEL.pdf', 'mime type': 'application/pdf', 'created at': '2025-09-10T01:43:11.251Z', 'modified at': '2025-08-08T17:10:16.000Z'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='VISITING US MIILITALY PERSONNEL \nApril 2025 \nA. PURPOSE \nVisit a US military personnel stationed in Japan or his/her dependent . \n \nB. Requirements（Details→https://www.ph.emb-japan

In [ ]:
documents[0].metadata['mime type']

In [ ]:
documents[0].metadata['file path'].split('/')

In [ ]:
documents[0].text_resource.text

In [3]:
import shutil
import os

cache_dir = os.path.expanduser("~/.llama_index")
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir, ignore_errors=True)


In [11]:
from llama_index.core.ingestion import IngestionPipeline, IngestionCache

cache = IngestionCache()
# pipeline = IngestionPipeline(cache=cache)

print(cache) # This might work in some older versions

nodes_key='nodes' collection='llama_cache' cache=<llama_index.core.storage.kvstore.simple_kvstore.SimpleKVStore object at 0x132435c10>
